In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [13]:
path = './chromedriver'
web = 'https://www.audible.com/search'

In [14]:
options = Options()
options.headless = False
# options.add_argument('window_size=1920x1080')
driver = webdriver.Chrome(path, options= options)
driver.get(web)
driver.maximize_window()

In [15]:
# Pagin
pagin = driver.find_element_by_xpath('.//ul[contains(@class, "pagingElements")]')
pages = pagin.find_elements_by_tag_name('li')
last_page = int(pages[-2].text)

In [16]:
book_title = []
book_author = []
book_length = []

current_page = 1

while current_page <= last_page:
    # time.sleep(3)
    container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "adbl-impression-container")))
    # container = driver.find_element_by_class_name("adbl-impression-container")
    products = WebDriverWait(container,5).until(EC.presence_of_all_elements_located((By.XPATH, './li')))
    # products = container.find_elements_by_xpath('./li')
    book_title += [product.find_element_by_xpath('.//h3[contains(@class, "bc-heading")]').text for product in products]
    book_author += [product.find_element_by_xpath('.//li[contains(@class, "authorLabel")]').text for product in products]
    book_length += [product.find_element_by_xpath('.//li[contains(@class, "runtimeLabel")]').text for product in products]
    current_page += 1
    try:
        next_page = driver.find_element_by_xpath('//span[contains(@class, "nextButton")]')
        next_page.click()
    except:
        pass
driver.quit()

In [ ]:
df_books = pd.DataFrame({'Title': book_title, 'Author': book_author, 'length': book_length})

df_books.to_csv('books_all_pages_explicit_wait.csv', index= False)